<a href="https://colab.research.google.com/github/farhanfuad35/titanic/blob/main/Titanic_775_alternative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up Kaggle Environment and Loading the Dataset

In [1]:
# Bash Scripts
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json   # 6: 4 + 2, Read & Write permission to myself

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
# Downloading Datasets
! kaggle competitions download -c titanic

test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# Importing Required Modules
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [4]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Data Imputation
## Age
For 'Age' I am going to use its mean value first to impute the data

In [5]:
# Impute 'Age' column with its mean value
data_train_cleaned = data_train
data_test_cleaned = data_test

na_val = data_train['Age'].mean()
print(na_val)
data_train_cleaned = data_train.fillna(value={'Age': na_val})

na_val = data_test['Age'].mean()
print(na_val)
data_test_cleaned = data_test_cleaned.fillna(value={'Age': na_val})

29.69911764705882
30.272590361445783


In [6]:
data_train_cleaned.head()
data_test_cleaned.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
y = data_train_cleaned['Survived']

In [8]:
# Convert Sex into Numerical Data
enc = OneHotEncoder(handle_unknown='ignore')
enc_data = pd.DataFrame(enc.fit_transform(data_train_cleaned[['Sex']]).toarray())
data_train_cleaned = data_train_cleaned.join(enc_data)
data_train_cleaned.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,0,1
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1.0,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.0,1.0


In [9]:
enc_data = pd.DataFrame(enc.fit_transform(data_test_cleaned[['Sex']]).toarray())
data_test_cleaned = data_test_cleaned.join(enc_data)
data_test_cleaned.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,0,1
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0,1.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1.0,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0,1.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0,1.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1.0,0.0


In [10]:
# Rename newly created numerical column names
data_train_cleaned = data_train_cleaned.rename(columns={0:'Male', 1:'Female'}, errors='raise')
data_test_cleaned = data_test_cleaned.rename(columns={0:'Male', 1:'Female'}, errors='raise')

data_train_cleaned.head()
data_test_cleaned.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Male,Female
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0,1.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1.0,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0,1.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0,1.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1.0,0.0


In [11]:
# Red: Create a new feature
# Store data in read-only variables
temp_train = data_train_cleaned
temp_test = data_test_cleaned

In [12]:
# Feature Extraction

data_train_cleaned = temp_train
data_train_cleaned['custom'] = (data_train_cleaned['Male']+1) * 2

# pClass
pclass = data_train_cleaned['Pclass']
pclass = pclass.replace({1:7, 2:5, 3:3})
data_train_cleaned['weighted_pclass'] = pclass
data_train_cleaned['custom'] = data_train_cleaned['custom']+pclass

# SibSp
sib = data_train_cleaned['SibSp']
sib = sib.replace({0:3, 1:5, 2:4, 3:2, 4:1, 5:0, 8:0})
data_train_cleaned['weighted_sib'] = sib
data_train_cleaned['custom'] = data_train_cleaned['custom']+sib

#Parch
parch = data_train_cleaned['Parch']
parch = parch.replace({0:3, 1:6, 2:5, 3:7, 4:0, 5:2, 6:0})
data_train_cleaned['weighted_parch'] = parch
data_train_cleaned['custom'] = data_train_cleaned['custom']+parch

# Age (16-47 lower weight)
data_train_cleaned['weighted_age'] = np.where( (data_train_cleaned['Age']>=16) & (data_train_cleaned['Age']<48), 1, 3)
data_train_cleaned['custom'] = data_train_cleaned['custom']+data_train_cleaned['weighted_age']

# Age-Sex feature
data_train_cleaned['age_sex'] = (data_train_cleaned['Male']+1) * 5
age = np.where( (data_train_cleaned['Age']>=16) & (data_train_cleaned['Age']<48), 1, 3)
data_train_cleaned['age_sex'] = data_train_cleaned['age_sex']+age

In [13]:
data_test_cleaned = temp_test
data_test_cleaned['custom'] = (data_test_cleaned['Male']+1) * 2

# pClass
pclass = data_test_cleaned['Pclass']
pclass = pclass.replace({1:7, 2:5, 3:3})
data_test_cleaned['weighted_pclass'] = pclass
data_test_cleaned['custom'] = data_test_cleaned['custom']+pclass

# SibSp
sib = data_test_cleaned['SibSp']
sib = sib.replace({0:3, 1:5, 2:4, 3:2, 4:1, 5:0, 8:0})
data_test_cleaned['weighted_sib'] = sib
data_test_cleaned['custom'] = data_test_cleaned['custom']+sib

#Parch
parch = data_test_cleaned['Parch']
parch = parch.replace({0:3, 1:6, 2:5, 3:7, 4:0, 5:2, 6:0})
data_test_cleaned['weighted_parch'] = parch
data_test_cleaned['custom'] = data_test_cleaned['custom']+parch

# Age (16-47 lower weight)
data_test_cleaned['weighted_age'] = np.where( (data_test_cleaned['Age']>=16) & (data_test_cleaned['Age']<48), 1, 3)
data_test_cleaned['custom'] = data_test_cleaned['custom']+data_test_cleaned['weighted_age']

# Age-Sex feature
data_test_cleaned['age_sex'] = (data_test_cleaned['Male']+1) * 5
age = np.where( (data_test_cleaned['Age']>=16) & (data_test_cleaned['Age']<48), 1, 3)
data_test_cleaned['age_sex'] = data_test_cleaned['age_sex']+age

In [14]:
# Remove Unnecessary Columns
X_train = data_train_cleaned.drop(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Ticket', 'Fare', 'Cabin', 'Embarked'], 1)
X_test = data_test_cleaned.drop(['PassengerId', 'Pclass', 'Name', 'Sex', 'Ticket', 'Fare', 'Cabin', 'Embarked'], 1)

In [15]:
print(X_train.head())
print(X_test.head())

    Age  SibSp  Parch  ...  weighted_parch  weighted_age  age_sex
0  22.0      1      0  ...               3             1      6.0
1  38.0      1      0  ...               3             1     11.0
2  26.0      0      0  ...               3             1     11.0
3  35.0      1      0  ...               3             1     11.0
4  35.0      0      0  ...               3             1      6.0

[5 rows x 11 columns]
    Age  SibSp  Parch  ...  weighted_parch  weighted_age  age_sex
0  34.5      0      0  ...               3             1      6.0
1  47.0      1      0  ...               3             1     11.0
2  62.0      0      0  ...               3             3      8.0
3  27.0      0      0  ...               3             1      6.0
4  22.0      1      1  ...               6             1     11.0

[5 rows x 11 columns]


In [16]:
# Train the Logistic Regression
logModel = LogisticRegression(max_iter=10000)
logModel.fit(X_train, y)

LogisticRegression(max_iter=10000)

In [17]:
# Predict test dataset

predictions = logModel.predict(X_test)

output = pd.DataFrame({'PassengerId': data_test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [18]:
# Submit via Kaggle API
! kaggle competitions submit titanic -f submission.csv -m "Added individual and combined"

100% 2.77k/2.77k [00:00<00:00, 5.51kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [19]:
# See Submissions
! kaggle competitions submissions titanic

fileName        date                 description                                               status    publicScore  privateScore  
--------------  -------------------  --------------------------------------------------------  --------  -----------  ------------  
submission.csv  2022-01-30 15:26:48  Added individual and combined                             complete  0.77511      None          
submission.csv  2022-01-30 15:12:55  Feature Count Updated                                     complete  0.77511      None          
submission.csv  2022-01-30 14:40:01  Feature Count Updated                                     complete  0.77511      None          
submission.csv  2022-01-30 11:39:36  Feature Count Updated                                     complete  0.76555      None          
submission.csv  2022-01-30 08:46:14  Kaggle API Submission. Trying with median imputation (2)  complete  0.77272      None          
submission.csv  2022-01-30 08:44:21  Kaggle API Submission. Trying wi